## Coursera Capstone Project

## The Battle of Two Cities

### The cities here in consideration are Delhi and Bombay

Mumbai and Delhi are the two most important metro cities in India. There has been a war for supremacy in terms of quality of life, jobs, education, entertainment and recreational facilities that these cities have to offer to its residents. I  attemp to analyze the neighborhoods in each of these two cities and try to understand what is popular in them and what they have to offer to someone who is contemplating to make a choice on seeking a life in either of the metro cities of India.


### The Business Problem 

The business problem in this study assumes that people who would be interested in this study are those who would like to create a projection of potential life and activities in these metro city neighborhoods if the subject moves to live in one of them. The decision to choose one over the other would depend on popular venues in the neighborhoods in each of these metro cities.

#### Importing all the necessary Libraries 

In [227]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from geopy.extra.rate_limiter import RateLimiter
import requests

About the data
It is a data provided by the idian government.
link:https://data.gov.in/catalog/all-india-pincode-directory?filters%5Bfield_catalog_reference%5D=85840&format=json&offset=0&limit=6&sort%5Bcreated%5D=desc
The data published by the government is on postal codes for all India would help me in the project.

In [3]:
df=pd.read_csv('all_india_PO_list_without_APS_offices_ver2_lat_long.csv')
df.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
0,Achalapur B.O,504273,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Rechini S.O,Mancherial H.O,NaN,NaN
1,Ada B.O,504293,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Asifabad,Adilabad,TELANGANA,NaN,Asifabad S.O,Mancherial H.O,NaN,NaN
2,Adegaon B.O,504307,B.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Boath,Adilabad,TELANGANA,NaN,Echoda S.O,Adilabad H.O,NaN,NaN
3,Adilabad Collectorate S.O,504001,S.O,Non-Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226703,NaN,Adilabad H.O,NaN,NaN
4,Adilabad H.O,504001,H.O,Delivery,Adilabad,Hyderabad,Andhra Pradesh,Adilabad,Adilabad,TELANGANA,08732-226738,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['officename', 'pincode', 'officeType', 'Deliverystatus', 'divisionname',
       'regionname', 'circlename', 'Taluk', 'Districtname', 'statename',
       'Telephone', 'Related Suboffice', 'Related Headoffice', 'longitude',
       'latitude'],
      dtype='object')

In [4]:
df.shape

(154797, 15)

In [5]:
df['regionname'].value_counts()

Muzaffarpur                5150
Hyderabad                  4674
South Karnataka            4594
North Karnataka            4487
Vijayawada                 4477
Jodhpur                    4457
Bhopal HQ                  4201
South Bengal               3955
Patna HQ                   3913
Aurangabad                 3882
Gorakhpur                  3700
Vadodara                   3686
Kurnool                    3680
Bhubaneswar HQ             3531
Tiruchy                    3505
Bareilly                   3504
Ajmer                      3502
Calcutta                   3183
Madurai                    3164
Raipur                     3150
Ranchi                     3097
Coimbatore                 3047
Agra                       3043
Rajkot                     3029
Lucknow  HQ                2992
Nagpur                     2931
Indore                     2874
Visakhapatnam              2781
Shimla HQ                  2781
Dehradun                   2715
Ambala  HQ                 2689
Chandiga

### from the loaded dataset we create two dataframes
    1. One for mumbai
    2. one for delhi
    First we make for mumbai

In [140]:
df_mumb=df[df['regionname']=='Mumbai']

In [141]:
df_mumb.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
81395,Antop Hill S.O,400037,S.O,Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24120290,NaN,Dadar H.O,NaN,NaN
81396,B P T Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-4100525,NaN,Dadar H.O,NaN,NaN
81397,B.P.Lane S.O,400003,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23421653,NaN,Chinchbunder H.O,NaN,NaN
81398,BEST STaff Quarters S.O,400012,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-24180776,NaN,Chinchbunder H.O,NaN,NaN
81399,C G S Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24097933,NaN,Dadar H.O,NaN,NaN


We add a new column to the dataset namely Address which will be ',' separated and will have 3 features which help us to determine the latitude adn longitude of the address

In [142]:
df_mumb['Address']=df_mumb['officename']+','+df_mumb['regionname']+','+df_mumb['statename']

c:\users\addy saish\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [143]:
df_mumb.shape

(1123, 16)

In [144]:
df_mumb.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude,Address
81395,Antop Hill S.O,400037,S.O,Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24120290,NaN,Dadar H.O,NaN,NaN,"Antop Hill S.O,Mumbai,MAHARASHTRA"
81396,B P T Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-4100525,NaN,Dadar H.O,NaN,NaN,"B P T Colony S.O,Mumbai,MAHARASHTRA"
81397,B.P.Lane S.O,400003,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-23421653,NaN,Chinchbunder H.O,NaN,NaN,"B.P.Lane S.O,Mumbai,MAHARASHTRA"
81398,BEST STaff Quarters S.O,400012,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,NaN,Mumbai,MAHARASHTRA,022-24180776,NaN,Chinchbunder H.O,NaN,NaN,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA"
81399,C G S Colony S.O,400037,S.O,Non-Delivery,Mumbai East,Mumbai,Maharashtra,Mumbai,Mumbai,MAHARASHTRA,022-24097933,NaN,Dadar H.O,NaN,NaN,"C G S Colony S.O,Mumbai,MAHARASHTRA"


In [145]:
df_mumb.drop(['officeType','Deliverystatus','divisionname','Taluk','Districtname','statename','Telephone','circlename','Related Suboffice','Related Headoffice'],axis=1,inplace=True)
df_mumb.head()

,officename,pincode,regionname,longitude,latitude,Address
81395,Antop Hill S.O,400037,Mumbai,NaN,NaN,"Antop Hill S.O,Mumbai,MAHARASHTRA"
81396,B P T Colony S.O,400037,Mumbai,NaN,NaN,"B P T Colony S.O,Mumbai,MAHARASHTRA"
81397,B.P.Lane S.O,400003,Mumbai,NaN,NaN,"B.P.Lane S.O,Mumbai,MAHARASHTRA"
81398,BEST STaff Quarters S.O,400012,Mumbai,NaN,NaN,"BEST STaff Quarters S.O,Mumbai,MAHARASHTRA"
81399,C G S Colony S.O,400037,Mumbai,NaN,NaN,"C G S Colony S.O,Mumbai,MAHARASHTRA"


In [146]:
df_mumb.rename(columns={'officename':'Neighborhood'},inplace=True)

In [147]:
df_mumb.drop(['Address'],axis=1,inplace=True)

In [149]:
df_mumb.drop(['latitude','longitude'],axis=1,inplace=True)

In [150]:
df_mumb.head()

,Neighborhood,pincode,regionname
81395,Antop Hill S.O,400037,Mumbai
81396,B P T Colony S.O,400037,Mumbai
81397,B.P.Lane S.O,400003,Mumbai
81398,BEST STaff Quarters S.O,400012,Mumbai
81399,C G S Colony S.O,400037,Mumbai


In [153]:
df_mumb=df_mumb[['pincode','Neighborhood','regionname']]
df_mumb.head()

,pincode,Neighborhood,regionname
81395,400037,Antop Hill S.O,Mumbai
81396,400037,B P T Colony S.O,Mumbai
81397,400003,B.P.Lane S.O,Mumbai
81398,400012,BEST STaff Quarters S.O,Mumbai
81399,400037,C G S Colony S.O,Mumbai


In [160]:
result1=df_mumb.groupby(['pincode','regionname'],sort=False).agg(','.join)
df_mumb=result1.reset_index()
df_mumb.head()

,pincode,regionname,Neighborhood
0,400037,Mumbai,"Antop Hill S.O,B P T Colony S.O,C G S Colony S..."
1,400003,Mumbai,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu..."
2,400012,Mumbai,"BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin..."
3,400009,Mumbai,"Chinchbunder H.O,Noor Baug S.O,Princess Dock S.O"
4,400033,Mumbai,"Cotton Exchange S.O,Kalachowki S.O,L B S N E c..."


Creating the dataframe having only values related to delhi

In [192]:
df_del=df[df['regionname']=='Delhi']

In [193]:
df_del.head()

,officename,pincode,officeType,Deliverystatus,divisionname,regionname,circlename,Taluk,Districtname,statename,Telephone,Related Suboffice,Related Headoffice,longitude,latitude
32383,Anand Vihar S.O,110092,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22157472,NaN,Krishna Nagar H.O,NaN,NaN
32384,Azad Nagar S.O (East Delhi),110051,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22093521,NaN,Krishna Nagar H.O,NaN,NaN
32385,Babarpur S.O (North East Delhi),110032,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,North East Delhi,DELHI,011-22829634,NaN,Jhilmil H.O,NaN,NaN
32386,Badarpur Khadar B.O,110090,B.O,Delivery,Delhi East,Delhi,Delhi,East Delhi,East Delhi,DELHI,NaN,Karawal Nagar S.O,Jhilmil H.O,NaN,NaN
32387,Balbir Nagar S.O,110032,S.O,Non-Delivery,Delhi East,Delhi,Delhi,NaN,East Delhi,DELHI,011-22320223,NaN,Jhilmil H.O,NaN,NaN


In [194]:
df_del.shape

(545, 15)

We add a new column to the dataset namely Address which will be ',' separated and will have 3 features which help us to determine the latitude adn longitude of the address

In [195]:
df_del.drop(['officeType','Deliverystatus','divisionname','Taluk','Districtname','statename','Telephone','circlename','Related Suboffice','Related Headoffice'],axis=1,inplace=True)
df_del.head()

,officename,pincode,regionname,longitude,latitude
32383,Anand Vihar S.O,110092,Delhi,NaN,NaN
32384,Azad Nagar S.O (East Delhi),110051,Delhi,NaN,NaN
32385,Babarpur S.O (North East Delhi),110032,Delhi,NaN,NaN
32386,Badarpur Khadar B.O,110090,Delhi,NaN,NaN
32387,Balbir Nagar S.O,110032,Delhi,NaN,NaN


In [196]:
df_del.drop(['latitude','longitude'],axis=1,inplace=True)

In [197]:
df_del.rename(columns={'officename':'Neighborhood'},inplace=True)

In [198]:
df_del=df_del[['pincode','Neighborhood','regionname']]
df_del.head()

,pincode,Neighborhood,regionname
32383,110092,Anand Vihar S.O,Delhi
32384,110051,Azad Nagar S.O (East Delhi),Delhi
32385,110032,Babarpur S.O (North East Delhi),Delhi
32386,110090,Badarpur Khadar B.O,Delhi
32387,110032,Balbir Nagar S.O,Delhi


In [199]:
result2=df_del.groupby(['pincode','regionname'],sort=False).agg(','.join)
df_del=result2.reset_index()
df_del.head()

,pincode,regionname,Neighborhood
0,110092,Delhi,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S..."
1,110051,Delhi,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr..."
2,110032,Delhi,"Babarpur S.O (North East Delhi),Balbir Nagar S..."
3,110090,Delhi,"Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed ..."
4,110053,Delhi,"Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O..."


In [200]:
nom=Nominatim(user_agent='myGeocoder')
geocode = RateLimiter(nom.geocode, min_delay_seconds=1)
df_del['Coordinates']=df_del['pincode'].apply(geocode)

In [201]:
df_del.head()

,pincode,regionname,Neighborhood,Coordinates
0,110092,Delhi,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...","(Dadri, Gautam Buddha Nagar, Uttar Pradesh, 11..."
1,110051,Delhi,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...","(Preet Vihar Tehsil, East Delhi, Delhi, 110051..."
2,110032,Delhi,"Babarpur S.O (North East Delhi),Balbir Nagar S...","(Navin Shahdara, Vivek Vihar Tehsil, Shahdara,..."
3,110090,Delhi,"Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed ...","(110090, Rua Nova Descoberta, Bolo de Noiva, N..."
4,110053,Delhi,"Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O...","(Babarpur, Shahdara Tehsil, Shahdara, Delhi, 1..."


here we adding a new column for latitude to the dataframe by defining a lambda function which will return the latitude from the coordinates column 

In [202]:
df_del['latitude']=df_del['Coordinates'].apply(lambda x: x.latitude if x!= None else None)

In [203]:
df_del['longitude']=df_del['Coordinates'].apply(lambda x: x.longitude if x!= None else None)

In [204]:
df_del.head()

,pincode,regionname,Neighborhood,Coordinates,latitude,longitude
0,110092,Delhi,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...","(Dadri, Gautam Buddha Nagar, Uttar Pradesh, 11...",28.551696,77.417594
1,110051,Delhi,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...","(Preet Vihar Tehsil, East Delhi, Delhi, 110051...",28.653165,77.286960
2,110032,Delhi,"Babarpur S.O (North East Delhi),Balbir Nagar S...","(Navin Shahdara, Vivek Vihar Tehsil, Shahdara,...",28.676479,77.285673
3,110090,Delhi,"Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed ...","(110090, Rua Nova Descoberta, Bolo de Noiva, N...",-8.016630,-34.926281
4,110053,Delhi,"Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O...","(Babarpur, Shahdara Tehsil, Shahdara, Delhi, 1...",28.691913,77.279820


Drop all the rows that reurned NaN in the latitude and longitude column

In [205]:
df_del=df_del.dropna()

In [207]:
df_del.head()

,pincode,regionname,Neighborhood,Coordinates,latitude,longitude
0,110092,Delhi,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...","(Dadri, Gautam Buddha Nagar, Uttar Pradesh, 11...",28.551696,77.417594
1,110051,Delhi,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...","(Preet Vihar Tehsil, East Delhi, Delhi, 110051...",28.653165,77.286960
2,110032,Delhi,"Babarpur S.O (North East Delhi),Balbir Nagar S...","(Navin Shahdara, Vivek Vihar Tehsil, Shahdara,...",28.676479,77.285673
3,110090,Delhi,"Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed ...","(110090, Rua Nova Descoberta, Bolo de Noiva, N...",-8.016630,-34.926281
4,110053,Delhi,"Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O...","(Babarpur, Shahdara Tehsil, Shahdara, Delhi, 1...",28.691913,77.279820


In [208]:
df_del.shape

(95, 6)

In [211]:
geocode = RateLimiter(nom.geocode, min_delay_seconds=1)
df_mumb['Coordinates']=df_mumb['pincode'].apply(geocode)

RateLimiter caught an error, retrying (0/2 tries). Called with (*(400032,), **{}).
Traceback (most recent call last):
  File "c:\users\addy saish\python\lib\urllib\request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "c:\users\addy saish\python\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1275, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1224, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "c:\users\addy saish\python\lib\http\client.py", line 1016, in _send_output
    self.send(msg)
  File "c:\users\addy saish\python\lib\http\client.py", line 956, in send
    self.connect()
  File "c:\users\addy saish\python\lib\http\client.py", line 1392, in connect
    server_hostname=server_hostname)
 

In [212]:
df_mumb.head()

,pincode,regionname,Neighborhood,Coordinates
0,400037,Mumbai,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...","(F/N Ward, Zone 2, Mumbai, Mumbai City, Mahara..."
1,400003,Mumbai,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...","(B Ward, Zone 1, Mumbai, Mumbai City, Maharash..."
2,400012,Mumbai,"BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin...","(F/S Ward, Zone 2, Mumbai, Mumbai City, Mahara..."
3,400009,Mumbai,"Chinchbunder H.O,Noor Baug S.O,Princess Dock S.O","(B Ward, Zone 1, Mumbai, Mumbai City, Maharash..."
4,400033,Mumbai,"Cotton Exchange S.O,Kalachowki S.O,L B S N E c...","(F/S Ward, Zone 2, Mumbai, Mumbai City, Mahara..."


In [213]:
df_mumb['latitude']=df_mumb['Coordinates'].apply(lambda x:x.latitude if x!=None else None)

In [214]:
df_mumb['longitude']=df_mumb['Coordinates'].apply(lambda x:x.longitude if x!=None else None)

In [216]:
df_mumb=df_mumb.dropna()

In [218]:
df_mumb.drop(['Coordinates'],axis=1,inplace=True)

In [219]:
df_del.drop(["Coordinates"],axis=1,inplace=True)

In [220]:
df_mumb.shape

(223, 5)

In [221]:
df_del.shape

(95, 5)

In [223]:
df_del.head()

,pincode,regionname,Neighborhood,latitude,longitude
0,110092,Delhi,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...",28.551696,77.417594
1,110051,Delhi,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",28.653165,77.286960
2,110032,Delhi,"Babarpur S.O (North East Delhi),Balbir Nagar S...",28.676479,77.285673
3,110090,Delhi,"Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed ...",-8.016630,-34.926281
4,110053,Delhi,"Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O...",28.691913,77.279820


In [222]:
df_mumb.head()

,pincode,regionname,Neighborhood,latitude,longitude
0,400037,Mumbai,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...",19.023074,72.867622
1,400003,Mumbai,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",18.951606,72.834797
2,400012,Mumbai,"BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin...",18.996311,72.842493
3,400009,Mumbai,"Chinchbunder H.O,Noor Baug S.O,Princess Dock S.O",18.959645,72.838526
4,400033,Mumbai,"Cotton Exchange S.O,Kalachowki S.O,L B S N E c...",19.003996,72.853620


In [54]:
address='Mumbai'
location=nom.geocode(address)
latitude_mum=location.latitude
longitude_mum=location.longitude
print('{},{}'.format(latitude_mum,longitude_mum))

18.9387711,72.8353355


In [55]:
address='Delhi'
location=nom.geocode(address)
latitude_del=location.latitude
longitude_del=location.longitude
print('{},{}'.format(latitude_del,longitude_del))

28.6517178,77.2219388


In [230]:
map_mum=folium.Map(location=[latitude_mum,longitude_mum],zoom_start=10)

for lat1,lon1,neighborhood,regioname in zip(df_mumb['latitude'],df_mumb['longitude'],df_mumb['Neighborhood'],df_mumb['regionname']):
    label='{},{}'.format(neighborhood,regioname)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat1,lon1],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_mum)
map_mum

In [231]:
map_del=folium.Map(location=[latitude_del,longitude_del],zoom_start=10)

for lat2,lon2,neighborhood,regioname in zip(df_del['latitude'],df_del['longitude'],df_del['Neighborhood'],df_del['regionname']):
    label='{},{}'.format(neighborhood,regioname)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat2,lon2],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_del)
map_del

In [245]:
CLIENT_ID='PRFI05SBWGTTQT2WCCN5FMBAIG0Y2INGQBFPPMDEU3J0GZDA'
CLIENT_SECRET='H0IO3LMRZKIGO20CV1WNJ1NXJ3H0SYSREGFDDJYWU12OZVBP'
VERSION='20180605'
radius=3000
LIMIT=100

In [246]:
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat1,
        lon1,
        radius,
        LIMIT        
        )

In [247]:
result_mum=requests.get(url1).json()
result_mum

{'meta': {'code': 200, 'requestId': '5e901a4b0de0d90023add8fd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Virār',
  'headerFullLocation': 'Virār',
  'headerLocationGranularity': 'city',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 19.48781362700003,
    'lng': 72.8487874980734},
   'sw': {'lat': 19.433813572999973, 'lng': 72.7916224019266}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e298f4a1850679fa413778f',
       'name': 'NATURALS ICE-CREAM',
       'location': {'address': 'AGASHI ROAD',
        'crossStreet': 'OPP. ICICI BANK',
        'lat': 19.447924337764427,
        'lng': 72.80131436803713,
        'labeledLatLngs': [{'label': 'display',
         

In [248]:
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat2,
        lon2,
        radius,
        LIMIT        
        )

In [249]:
result_del=requests.get(url2).json()
result_del

{'meta': {'code': 200, 'requestId': '5e90199ba2e538001b403ced'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Delhi',
  'headerFullLocation': 'Delhi',
  'headerLocationGranularity': 'city',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 28.634382452000025,
    'lng': 77.1330586490683},
   'sw': {'lat': 28.58038239799997, 'lng': 77.0716645009317}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d29ded76e27a14371db2824',
       'name': 'Manjeet Chicken',
       'location': {'address': 'Fatehnagar, Janakpuri',
        'lat': 28.631451808276633,
        'lng': 77.09793841882282,
        'labeledLatLngs': [{'label': 'display',
          'lat': 28.631451808276633,
       

In [260]:
def getnearbyvenues(names,latitudes,longitudes,radius=500):
    venue_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius,
        LIMIT        
        )
        results=requests.get(url).json()['response']['groups'][0]['items']
        venue_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
    nearby_venue=pd.DataFrame([item for venuelist in venue_list for item in venuelist])
    nearby_venue.columns=['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venue

In [261]:
mumbai_venue=getnearbyvenues(names=df_mumb['Neighborhood'],latitudes=df_mumb['latitude'],longitudes=df_mumb['longitude'])

Antop Hill S.O,B P T Colony S.O,C G S Colony S.O,Wadala Truck Terminal S.O
B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Null Bazar S.O
BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin Institute S.O,Lal Baug S.O,Parel Naka S.O,Parel Rly Work Shop S.O,Parel S.O
Chinchbunder H.O,Noor Baug S.O,Princess Dock S.O
Cotton Exchange S.O,Kalachowki S.O,L B S N E collage S.O,Reay Road S.O,Tank Road S.O
Dadar Colony S.O,Dadar H.O,Naigaon S.O (Mumbai)
Dockyard Road S.O,Mazgaon Dock S.O,Mazgaon Road S.O,Mazgaon S.O,V K Bhavan S.O
Kidwai Nagar S.O (Mumbai),Wadala Rs S.O,Wadala S.O
Sewri S.O
V J B Udyan S.O
A I Staff Colony S.O,Santacruz P&t Colony S.O
Airport S.O (Mumbai),International Airport S.O,Sahar P & T Colony S.O,Sahargaon B.O
Andheri East S.O,Nagardas Road S.O
Andheri H.O,Azad Nagar S.O (Mumbai)
Andheri Railway Station S.O,H.M.P. School S.O
Audit Bhavan S.O,B.N. Bhavan S.O,Bandra(East) S.O,Government Colony S.O,Kherwadi S.O
Bandra West S.O
Chakala Midc S.O
Danda S.O,Khar Colony S.O,Khar Delivery 

Barasgaon B.O,Kumbheshivthar B.O,Mazeri B.O,Padvi B.O,Taliye B.O,Varandh S.O
Bhale B.O,Borwadi B.O,Jite B.O,Koste Khurd B.O,Nizampur S.O (Raigarh(MH)),Shirawali B.O
Bhave B.O,Birwadi S.O,Dahivad B.O,Dapoli B.O,Dhamne B.O,Jite B.O,Mangrun B.O,Mohot B.O,Nigade B.O,Wagholi B.O,Waki Budruk B.O,Walan Budruk B.O,Walan Khurd B.O
Bhonang B.O,Borghar Haveli B.O,Mandad B.O,Pitsai B.O,Rahatad B.O,Tala S.O (Raigarh(MH)),Talegaon B.O,Tokarde B.O
Bhoste B.O,Galsure B.O,Hareshwar B.O,Jasawali B.O,Ranawali B.O,Saigaon B.O,Shriwardhan S.O,Vadghar B.O,Vadshetvave B.O
Borlimandla B.O,Korlai B.O,Revdanda S.O,Vikram Baug B.O,Walke B.O
Chikhlap B.O,Gondghar B.O,Kelte B.O,Khamgaon B.O,Kharsai B.O,Kolwat B.O,Majgaon Tamhane B.O,Mendadi B.O,Mhasla S.O,Pabhre B.O,Salvinde B.O,Talavade B.O,Varvatane B.O
Chinchwali Tarfe Atone B.O,Devkanhe B.O,Khamb B.O,Kolad S.O,Pugaon B.O,Talawali Terfe Diwali B.O
Dabhol B.O,Dasgaon S.O,TolBudruk B.O,Veer B.O
Dahivali B.O,Degaon B.O,Morba S.O
Dighi S.O,Kudgaon B.O
Mahagaon B.O,

In [262]:
delhi_venue=getnearbyvenues(names=df_del['Neighborhood'],latitudes=df_del['latitude'],longitudes=df_del['longitude'])

Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S.O (East Delhi),Mandawali Fazalpur S.O,Nirman Vihar S.O,Shakarpur S.O,Surajmal Vihar S.O,Yozna Vihar S.O
Azad Nagar S.O (East Delhi),Govind Pura S.O,Krishna Nagar H.O,Ram Nagar S.O (East Delhi)
Babarpur S.O (North East Delhi),Balbir Nagar S.O,Bhola Nath Nagar S.O,Distt. Court (KKD) S.O,Goverdhan Bihari Colony S.O,Loni Road S.O,Man Sarovar Park S.O,Rohtash Nagar S.O,Shahdara Mandi S.O,Shahdara S.O,Shivaji Park S.O (East Delhi),Telewara S.O,Vishwas Nagar S.O
Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed Bhagat Singh Colony S.O,Shriram Colony Rajeev Nagar S.O,Sonia Vihar S.O
Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O,Ghonda B.O,Jafrabad S.O (East Delhi),Jagjit Nagar S.O,Maujpur S.O,New Usmanpur B.O,Seelampur S.O,Yamuna Vihar S.O
Chilla B.O,Himmatpuri S.O,Kalyanpuri S.O,Kalyanvas S.O,Mayur Vihar Ph-I S.O,Patparganj S.O,Trilok Puri S.O
Dayalpur B.O,Gokal Puri S.O,Johripur B.O,Khazuri Khas B.O,Sabhapur B.O
Dilshad Garden S.O,G.T.B. Hospital S

Ashok Nagar S.O (West Delhi),Chand Nagar S.O,Chaukhandi B.O,Fateh Nagar S.O,Khyala Phase -  I S.O,Khyala Phase - II S.O,M.B.S. Nagar S.O,Mahabir Nagar S.O,Tilak Nagar East S.O,Tilak Nagar S.O (West Delhi),Vikas Puri S.O,Vishnu Garden S.O
Ashoka Park Extn. S.O,Punjabi Bagh S.O,Punjabi Bagh Sec - III S.O,Shivaji Park S.O (West Delhi)
Badusarai B.O,Chhawla S.O,DC Goyla B.O,Kangan Heri B.O
Bagdola B.O,Barthal B.O,Dhulsiras B.O,Palam Extn (Harijan Basti) S.O,Raj Nagar - II S.O
Bakkarwala B.O,Hirankudna B.O,Mundka B.O,Nangloi - II B.O,Nangloi - III S.O,Nangloi S.O,Nilothi B.O,Ranhola B.O,Tikri Kalan B.O
Baprola B.O,Daulatpur B.O,Dichaun Kalan B.O,Dindarpur B.O,Jhatikara B.O,Mitraon B.O,Najafgarh S.O,Pandwala Kalan B.O,Papravat B.O,Rewla Khanpur B.O,Surehra B.O,Tilangpur Kotla B.O
CRPF Jharoda Kalan S.O,Jharoda Kalan B.O
D. K. Mohan Garden S.O,Hastal Village B.O,Jeevan Park B.O,Matiala B.O,Uttam Nagar S.O
D.E.S.U. Colony S.O,Jail Road S.O (West Delhi),Janakpuri A-3 S.O,Janakpuri B-1 S.O,Janak

KeyError: 'groups'

In [263]:
mumbai_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...",19.023074,72.867622,Prince,19.022008,72.869424,Smoke Shop
1,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...",19.023074,72.867622,Palkhi Restaurant,19.022163,72.863034,Indian Restaurant
2,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",18.951606,72.834797,Gulshan-E-Iran,18.948118,72.835427,Middle Eastern Restaurant
3,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",18.951606,72.834797,Bhagat Tarachand Restaurant,18.951802,72.830486,Indian Restaurant
4,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",18.951606,72.834797,Patel Restaurant,18.949798,72.834655,Restaurant


In [264]:
delhi_venue.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...",28.551696,77.417594,Caliber India,28.549959,77.411556,Clothing Store
1,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",28.653165,77.286960,Domino's Pizza,28.652000,77.278000,Pizza Place
2,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",28.653165,77.286960,Advance bakery,28.654662,77.287489,Bakery
3,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",28.653165,77.286960,Lal quarter,28.658959,77.282260,Accessories Store
4,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",28.653165,77.286960,Pizza Hut Krishna Nagar,28.660610,77.287529,Pizza Place


In [265]:
mumbai_venue.shape

(3239, 7)

In [266]:
mumbai_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"A I Staff Colony S.O,Santacruz P&t Colony S.O",18,18,18,18,18,18
"Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighode B.O,Gavan B.O,Jasai B.O,Jci Kamothe B.O,Jui B.O,Kelevane B.O,Khanda Colony S.O,Koproli B.O,Kundevahal B.O,Lodhivali B.O,Morbe B.O,Nere B.O,Nhava B.O,Panvel City S.O,Panvel H.O,Pargaon B.O,Somatane B.O,Ulwa B.O,Vahal B.O,Vaje B.O,Vasheni B.O,Wavandhal B.O,Wavarle B.O",1,1,1,1,1,1
"Adavale Budruk B.O,Borghar B.O,Chambhargani B.O,Deopur B.O,Devla B.O,Dhamandevi B.O,Divil B.O,Golegani B.O,Kapade Budruk B.O,Kondhavi B.O,Kotwal Budruk B.O,Lohare B.O,Mahargul B.O,Ombali B.O,Paithan B.O,Palchil B.O,Poladpur S.O,Sadavali B.O,Sakhar B.O,Turbhe Khurd B.O,Wakan B.O",1,1,1,1,1,1
Additional Ambernath S.O,4,4,4,4,4,4
"Adgaon B.O,Borlipanchatan S.O,Sarve B.O,Velas B.O,Wadwali B.O",1,1,1,1,1,1
"Agarwadi B.O,Dahisar tymanor B.O,Datiware B.O,Dhekale B.O,Edwan B.O,Navghar B.O,Pargaon B.O,Tandulwadi B.O,Tembhikhodave B.O,Umbarpada S.O,Usarani B.O,Virathan Budruk B.O",3,3,3,3,3,3
"Aghai B.O,Alyani B.O,Ambarje B.O,Andad B.O,Asangaon B.O,Atgaon B.O,Aware B.O,Chande B.O,Chondhe Colony B.O,Dalkhan B.O,Dhasai B.O,Dolkhamb B.O,Gunde B.O,Kalamgaon B.O,Kharade B.O,Khardi B.O,Khutghar B.O,Lenad Khurd B.O,Nandgaon B.O,Piwali B.O,Sakadbav B.O,Sakurli B.O,Saralambe B.O,Sawaroli B.O,Shahapur S.O (Thane),Shenwa B.O,Shere B.O,Shirgaon B.O,Tansa B.O,Tembhurli B.O,Vaitarna B.O,Vashind E.D. B.O,Vehele B.O,Vehloli B.O,Vehloli B.O",3,3,3,3,3,3
"Agripada S.O,BPC Jacob Circle S.O,Chinchpokli S.O,Haines Road S.O,Jacob Circle S.O",3,3,3,3,3,3
"Ainghar B.O,Pipenagar S.O",1,1,1,1,1,1


#### lets find out how many unique categories can be curated from all the returned venues

In [272]:
print('there are {} unique categories'.format(len(mumbai_venue['Venue Category'].unique())))

there are 224 unique categories


### Analyzing Each Neighborhood

In [274]:
mum_onehot=pd.get_dummies(mumbai_venue['Venue Category'],prefix=' ',prefix_sep='')
mum_onehot['Neighborhood']=mumbai_venue['Neighborhood']#add neighborhood column back to dataframe

# shifting neighborhood to the first colum
fixed_column=[mum_onehot.columns[-1]]+list(mum_onehot.columns[:-1])
mum_onehot=mum_onehot[fixed_column]
mum_onehot.head()


,Neighborhood,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Train Station,Tram Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Zoo
0,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [275]:
mum_onehot.shape

(3239, 225)

In [276]:
mum_grouped=mum_onehot.groupby('Neighborhood').mean().reset_index()
mum_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Train Station,Tram Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Zoo
0,"A I Staff Colony S.O,Santacruz P&t Colony S.O",0.0,0.0,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,...,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
1,"Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighod...",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Adavale Budruk B.O,Borghar B.O,Chambhargani B....",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Additional Ambernath S.O,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Adgaon B.O,Borlipanchatan S.O,Sarve B.O,Velas ...",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Returing  the top 5 venues of Each Neighborhood

In [277]:
num_top_venue=5
for hood in mum_grouped['Neighborhood']:
    print('-----'+hood+'-----')
    temp=mum_grouped[mum_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venue))
    print('\n')

-----A I Staff Colony S.O,Santacruz P&t Colony S.O-----
               venue  freq
0         Restaurant  0.17
1   Football Stadium  0.06
2                Gym  0.06
3        Coffee Shop  0.06
4             Lounge  0.06


-----Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighode B.O,Gavan B.O,Jasai B.O,Jci Kamothe B.O,Jui B.O,Kelevane B.O,Khanda Colony S.O,Koproli B.O,Kundevahal B.O,Lodhivali B.O,Morbe B.O,Nere B.O,Nhava B.O,Panvel City S.O,Panvel H.O,Pargaon B.O,Somatane B.O,Ulwa B.O,Vahal B.O,Vaje B.O,Vasheni B.O,Wavandhal B.O,Wavarle B.O-----
                            venue  freq
0   Paper / Office Supplies Store   1.0
1                             ATM   0.0
2                    Noodle House   0.0
3                        Mountain   0.0
4                   Movie Theater   0.0


-----Adavale Budruk B.O,Borghar B.O,Chambhargani B.O,Deopur B.O,Devla B.O,Dhamandevi B.O,Divil B.O,Golegani B.O,Kapade Budruk B.O,Kondhavi B.O,Kotwal Budruk B.O,Lohare B.O,Mahargul B.O,Ombali B.O,Paithan B.O,Palch

                venue  freq
0   Indian Restaurant  0.31
1   Convenience Store  0.06
2        Dessert Shop  0.06
3   Electronics Store  0.06
4          Restaurant  0.06


-----BARC S.O-----
                venue  freq
0                Café  0.38
1         Supermarket  0.25
2   Cretan Restaurant  0.12
3        Dessert Shop  0.12
4    Greek Restaurant  0.12


-----BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin Institute S.O,Lal Baug S.O,Parel Naka S.O,Parel Rly Work Shop S.O,Parel S.O-----
                   venue  freq
0      Indian Restaurant  0.22
1             Restaurant  0.11
2     Chinese Restaurant  0.11
3                  Hotel  0.11
4   Gym / Fitness Center  0.11


-----Badlapur E.D. B.O,Chargaon B.O,Kulgaon S.O,Manjarli B.O,Pimpaloli B.O,Vangani B.O,Yeranjad B.O-----
            venue  freq
0   Grocery Store   1.0
1             ATM   0.0
2          Office   0.0
3   Movie Theater   0.0
4   Moving Target   0.0


-----Balkum S.O,Shrirangnagar B.O,Tapasenagar B.O-----
            ve

                venue  freq
0   Mobile Phone Shop   1.0
1                 ATM   0.0
2     Motorcycle Shop   0.0
3       Movie Theater   0.0
4       Moving Target   0.0


-----Chitalsar Manpada B.O,Sandozbaugh S.O-----
            venue  freq
0          Bakery  0.12
1           Hotel  0.12
2     Supermarket  0.12
3      Restaurant  0.06
4   Movie Theater  0.06


-----Chunabhatti S.O,Raoli Camp S.O,Sion S.O,Transit Camp S.O-----
                            venue  freq
0   Vegetarian / Vegan Restaurant  0.25
1                    Dance Studio  0.25
2               Indian Restaurant  0.25
3               Electronics Store  0.25
4                             ATM  0.00


-----Cotton Exchange S.O,Kalachowki S.O,L B S N E collage S.O,Reay Road S.O,Tank Road S.O-----
          venue  freq
0     Pool Hall  0.25
1      Pharmacy  0.25
2   Record Shop  0.25
3   Auto Garage  0.25
4           ATM  0.00


-----Cumballa Hill S.O,Cumballa Sea Face S.O,Dr Deshmukh Marg S.O,Gowalia Tank S.O-----
          

4           Pub  0.05


-----J.B. Nagar S.O,Marol Bazar S.O,Marol Naka S.O-----
                          venue  freq
0                         Plaza  0.14
1                         Hotel  0.14
2                   Bus Station  0.14
3                  Tram Station  0.14
4   Eastern European Restaurant  0.14


-----Jambhul B.O,Netaji Bazar S.O-----
                venue  freq
0           Multiplex   1.0
1                 ATM   0.0
2   Accessories Store   0.0
3            Mountain   0.0
4       Movie Theater   0.0


-----Jekegram S.O-----
                   venue  freq
0   Gym / Fitness Center  0.12
1            Pizza Place  0.12
2            Supermarket  0.08
3                 Bakery  0.08
4                    Gym  0.08


-----Jogeshwari East S.O-----
                   venue  freq
0   Fast Food Restaurant  0.67
1          Moving Target  0.33
2                    ATM  0.00
3          Historic Site  0.00
4          Movie Theater  0.00


-----Jogeshwari West S.O,Oshiwara S.O-----
         



-----New Prabhadevi Road S.O,Prabhadevi S.O-----
                venue  freq
0   Indian Restaurant  0.21
1                Café  0.11
2   Electronics Store  0.11
3         Pizza Place  0.05
4             Theater  0.05


-----O.E.Ambernath S.O-----
                 venue  freq
0                  ATM  0.25
1   Chinese Restaurant  0.25
2            Multiplex  0.25
3           Restaurant  0.25
4   Photography Studio  0.00


-----Padgha Extension Counter,Padgha S.O-----
                venue  freq
0               Plaza   0.5
1   Indian Restaurant   0.5
2              Office   0.0
3       Movie Theater   0.0
4       Moving Target   0.0


-----Powai Iit S.O-----
                 venue  freq
0           Restaurant  0.15
1                 Café  0.12
2   Italian Restaurant  0.08
3             Bus Stop  0.04
4   Spanish Restaurant  0.04


-----Psm Colony S.O,Vikhroli S.O-----
          venue  freq
0          Café  0.13
1    Restaurant  0.06
2           Pub  0.05
3           Bar  0.05
4   Coffee 

### Lets put into the DataFrame

first writing a func to sort venue in descending order

In [278]:
def return_most_common_venue(row,num_top_venue):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venue]

In [280]:
num_top_venue=10
indicators=['st','nd','rd']
columns=['Neighborhood']
for ind in np.arange(num_top_venue):
    try:
        columns.append("{}{} most common venue".format(ind+1,indicators[ind]))
    except:
        columns.append('{}th most common venue'.format(ind+1))
neighborhood_venue_sorted_mum=pd.DataFrame(columns=columns)
neighborhood_venue_sorted_mum['Neighborhood']=mum_grouped['Neighborhood']

for ind in np.arange(mum_grouped.shape[0]):
    neighborhood_venue_sorted_mum.iloc[ind,1:]=return_most_common_venue(mum_grouped.iloc[ind,:],num_top_venue)
neighborhood_venue_sorted_mum.head()
    

,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,"A I Staff Colony S.O,Santacruz P&t Colony S.O",Restaurant,Gym,Hotel,Coffee Shop,Pub,Recreation Center,Café,Salon / Barbershop,Lounge,Beer Garden
1,"Adai B.O,Awre B.O,Chawk B.O,Chirner B.O,Dighod...",Paper / Office Supplies Store,Zoo,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
2,"Adavale Budruk B.O,Borghar B.O,Chambhargani B....",Indian Restaurant,Zoo,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
3,Additional Ambernath S.O,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Zoo,Dog Run,Field,Farmers Market,Farm,Falafel Restaurant,Electronics Store
4,"Adgaon B.O,Borlipanchatan S.O,Sarve B.O,Velas ...",Bus Station,Zoo,Cosmetics Shop,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store


In [281]:
delhi_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"505 A B Workshop S.O,A F Palam S.O,Aps Colony B.O,Bazar Road S.O,C.V.D. S.O,COD S.O (South West Delhi),Delhi Cantt S.O,Dhaula Kuan S.O,Kirby Place S.O,Pinto Park S.O,R R Hospital S.O,Signal Enclave S.O,Station Road S.O (South West Delhi),Subroto Park S.O",100,100,100,100,100,100
"A F Rajokari S.O,Rajokari B.O",1,1,1,1,1,1
"A.G.C.R. S.O,Ajmeri Gate Extn. S.O,Darya Ganj S.O,Gandhi Smarak Nidhi S.O,I.P.Estate S.O,Indraprastha H.O,Minto Road S.O",18,18,18,18,18,18
"A.K.Market S.O,Multani Dhanda S.O,Pahar Ganj S.O,Swami Ram Tirth Nagar S.O",18,18,18,18,18,18
"Abul Fazal Enclave-I S.O,Jamia Nagar S.O,New Friends Colony S.O,Sukhdev Vihar S.O,Zakir Nagar S.O",6,6,6,6,6,6
"Adrash Nagar S.O,Bhalaswa B.O,Jahangir Puri A Block S.O,Jahangir Puri D Block S.O,Jahangir Puri H Block S.O,N.S.Mandi S.O",5,5,5,5,5,5
"Air Force Station Tugalkabad S.O,BSF Camp Tigri S.O,Dakshinpuri Phase-I S.O,Dakshinpuri Phase-II S.O,Dakshinpuri Phase-III S.O,Deoli B.O,Dr. Ambedkar Nagar S.O (South Delhi),Hamdard Nagar S.O,Khanpur S.O (South Delhi),Pushpa Bhawan S.O,Talimabad S.O",7,7,7,7,7,7
"Alaknanda S.O,Chittranjan Park S.O,Kalkaji H.O,Nehru Place S.O",45,45,45,45,45,45
"Ali B.O,Madanpur Khadar B.O,Sarita Vihar S.O",42,42,42,42,42,42


In [282]:
print('There are {} unique venue category'.format(len(delhi_venue['Venue Category'].unique())))

There are 219 unique venue category


In [285]:
del_onehot=pd.get_dummies(delhi_venue['Venue Category'],prefix=' ',prefix_sep='')
del_onehot['Neighborhood']=delhi_venue["Neighborhood"]

fixed_column1=[del_onehot.columns[-1]]+list(del_onehot.columns[:-1])
del_onehot=del_onehot[fixed_column1]
del_onehot.head()

,Neighborhood,ATM,Acai House,Accessories Store,Advertising Agency,African Restaurant,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,...,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint
0,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [286]:
del_onehot.shape

(1551, 220)

In [287]:
del_grouped=del_onehot.groupby('Neighborhood').mean().reset_index()
del_grouped.head()

,Neighborhood,ATM,Acai House,Accessories Store,Advertising Agency,African Restaurant,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,...,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint
0,"505 A B Workshop S.O,A F Palam S.O,Aps Colony ...",0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.01
1,"A F Rajokari S.O,Rajokari B.O",1.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
2,"A.G.C.R. S.O,Ajmeri Gate Extn. S.O,Darya Ganj ...",0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3,"A.K.Market S.O,Multani Dhanda S.O,Pahar Ganj S...",0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
4,"Abul Fazal Enclave-I S.O,Jamia Nagar S.O,New F...",0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00


In [289]:
num_top_venue2=5
for hood in del_grouped['Neighborhood']:
    print('-----'+hood+'-----')
    temp=del_grouped[del_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venue))
    print('**************************')

-----505 A B Workshop S.O,A F Palam S.O,Aps Colony B.O,Bazar Road S.O,C.V.D. S.O,COD S.O (South West Delhi),Delhi Cantt S.O,Dhaula Kuan S.O,Kirby Place S.O,Pinto Park S.O,R R Hospital S.O,Signal Enclave S.O,Station Road S.O (South West Delhi),Subroto Park S.O-----
                        venue  freq
0                  Restaurant  0.07
1                         Bar  0.04
2                     Gay Bar  0.04
3                      Bakery  0.04
4   Latin American Restaurant  0.04
**************************
-----A F Rajokari S.O,Rajokari B.O-----
                             venue  freq
0                              ATM   1.0
1                     Noodle House   0.0
2                    Moving Target   0.0
3               Mughlai Restaurant   0.0
4   Multicuisine Indian Restaurant   0.0
**************************
-----A.G.C.R. S.O,Ajmeri Gate Extn. S.O,Darya Ganj S.O,Gandhi Smarak Nidhi S.O,I.P.Estate S.O,Indraprastha H.O,Minto Road S.O-----
                venue  freq
0   Indian Restauran

                   venue  freq
0      Indian Restaurant  0.17
1   Fast Food Restaurant  0.17
2            Pizza Place  0.08
3     Light Rail Station  0.08
4             Food Truck  0.08
**************************
-----Azad Nagar S.O (East Delhi),Govind Pura S.O,Krishna Nagar H.O,Ram Nagar S.O (East Delhi)-----
                venue  freq
0         Pizza Place   0.4
1   Accessories Store   0.2
2                 Gym   0.2
3              Bakery   0.2
4        Noodle House   0.0
**************************
-----Babarpur S.O (North East Delhi),Balbir Nagar S.O,Bhola Nath Nagar S.O,Distt. Court (KKD) S.O,Goverdhan Bihari Colony S.O,Loni Road S.O,Man Sarovar Park S.O,Rohtash Nagar S.O,Shahdara Mandi S.O,Shahdara S.O,Shivaji Park S.O (East Delhi),Telewara S.O,Vishwas Nagar S.O-----
                            venue  freq
0              Light Rail Station  0.50
1   Vegetarian / Vegan Restaurant  0.25
2                            Park  0.25
3         North Indian Restaurant  0.00
4              M

             venue  freq
0             Café  0.25
1    Grocery Store  0.25
2              Gym  0.25
3   Clothing Store  0.25
4              ATM  0.00
**************************
-----Dabri B.O,Indira Park B.O,Nasirpur B.O,Palam Village S.O-----
                venue  freq
0                 ATM  0.25
1    Department Store  0.25
2   Indian Restaurant  0.25
3                 Gym  0.25
4        Noodle House  0.00
**************************
-----Dada Ghosh Bhawan S.O,Patel Nagar East S.O,Patel Nagar S.O (Central Delhi),Patel Nagar South S.O,Patel Nagar West S.O-----
                venue  freq
0    Asian Restaurant  0.33
1           Gastropub  0.17
2   Electronics Store  0.17
3    Sushi Restaurant  0.17
4          Restaurant  0.17
**************************
-----Dargah Sharif S.O,Hazrat Nizamuddin S.O-----
            venue  freq
0   Moving Target  0.11
1   Train Station  0.11
2            Café  0.11
3     Flea Market  0.11
4      Food Court  0.11
**************************
-----Dayalpur B.O

                   venue  freq
0   Fast Food Restaurant  0.09
1                   Café  0.09
2      Indian Restaurant  0.09
3            Pizza Place  0.07
4              BBQ Joint  0.04
**************************
-----Jawala Heri B.O,Madipur Slum Quarter S.O,Madipur Village S.O,Paschim Vihar B Block S.O,Paschim Vihar S.O,Peeragarhi B.O-----
                venue  freq
0              Bakery  0.25
1    Department Store  0.25
2   Indian Restaurant  0.25
3              Garden  0.25
4                 ATM  0.00
**************************
-----Jwala Puri S.O,Sunder Vihar S.O-----
             venue  freq
0      Pizza Place  0.11
1       Shoe Store  0.11
2   Breakfast Spot  0.11
3   Ice Cream Shop  0.11
4        Juice Bar  0.11
**************************
-----Kair B.O,Khaira B.O,Naraina Industrial Estate H.O,Naraina Village S.O-----
                   venue  freq
0                  Hotel   0.3
1            Pizza Place   0.2
2      Indian Restaurant   0.1
3              Multiplex   0.1
4   Fast

In [290]:
neighborhood_venue_sorted_del=pd.DataFrame(columns=columns)
neighborhood_venue_sorted_del['Neighborhood']=del_grouped['Neighborhood']
num_top_venue2=10
for ind in np.arange(del_grouped.shape[0]):
    neighborhood_venue_sorted_del.iloc[ind,1:]=return_most_common_venue(del_grouped.iloc[ind,:],num_top_venue2)
neighborhood_venue_sorted_del.head()

,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,"505 A B Workshop S.O,A F Palam S.O,Aps Colony ...",Restaurant,Gay Bar,Bakery,Latin American Restaurant,Arepa Restaurant,Bar,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Steakhouse,Burger Joint
1,"A F Rajokari S.O,Rajokari B.O",ATM,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant
2,"A.G.C.R. S.O,Ajmeri Gate Extn. S.O,Darya Ganj ...",Indian Restaurant,Hotel,Restaurant,Hostel,History Museum,Cricket Ground,Stadium,Road,Historic Site,Asian Restaurant
3,"A.K.Market S.O,Multani Dhanda S.O,Pahar Ganj S...",Hotel,Pizza Place,Motel,Fast Food Restaurant,Restaurant,Coffee Shop,Arts & Crafts Store,Miscellaneous Shop,Hostel,Food Truck
4,"Abul Fazal Enclave-I S.O,Jamia Nagar S.O,New F...",Indian Restaurant,Athletics & Sports,Coffee Shop,Gym / Fitness Center,Boat or Ferry,Hotel Pool,Farmers Market,Food Truck,Food Stand,Food Court


In [291]:
neighborhood_venue_sorted_del.shape

(93, 11)

In [292]:
neighborhood_venue_sorted_mum.shape

(164, 11)

### Clustering

In [295]:
mum_group_cluster=mum_grouped.drop('Neighborhood',1)
model1=KMeans(n_clusters=5,random_state=5).fit(mum_group_cluster)
model1.labels_

array([0, 0, 2, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 2, 0,
       0, 0, 3, 1, 0, 0, 0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       3, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 0, 0, 2])

In [298]:
neighborhood_venue_sorted_mum.insert(0,'Cluster_Label',model1.labels_)
mum_merged=df_mumb
mum_merged=mum_merged.join(neighborhood_venue_sorted_mum.set_index('Neighborhood'),on='Neighborhood')
mum_merged.head()

,pincode,regionname,Neighborhood,latitude,longitude,Cluster_Label,Cluster Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,400037,Mumbai,"Antop Hill S.O,B P T Colony S.O,C G S Colony S...",19.023074,72.867622,2.0,2.0,Indian Restaurant,Smoke Shop,Zoo,Doner Restaurant,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,400003,Mumbai,"B.P.Lane S.O,Mandvi S.O (Mumbai),Masjid S.O,Nu...",18.951606,72.834797,2.0,2.0,Indian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Chinese Restaurant,Café,Restaurant,BBQ Joint,Indian Sweet Shop,Dessert Shop,Electronics Store
2,400012,Mumbai,"BEST STaff Quarters S.O,Chamarbaug S.O,Haffkin...",18.996311,72.842493,0.0,0.0,Indian Restaurant,Hotel,Coffee Shop,Roof Deck,Lounge,Chinese Restaurant,Restaurant,Gym / Fitness Center,Eastern European Restaurant,Electronics Store
3,400009,Mumbai,"Chinchbunder H.O,Noor Baug S.O,Princess Dock S.O",18.959645,72.838526,0.0,0.0,Harbor / Marina,Indian Restaurant,Furniture / Home Store,Café,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store
4,400033,Mumbai,"Cotton Exchange S.O,Kalachowki S.O,L B S N E c...",19.003996,72.853620,0.0,0.0,Pharmacy,Pool Hall,Record Shop,Auto Garage,Zoo,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store


In [304]:
mum_merged=mum_merged.drop('Cluster Label',1)

In [305]:
mum_merged=mum_merged.dropna()

In [306]:
mum_merged['Cluster_Label']=mum_merged.Cluster_Label.astype(int)


In [315]:
map_cluster_mum=folium.Map(location=[latitude_mum,longitude_mum],zoom_start=10)
x=np.arange(5)
ys=[i+x+(i*x)**2 for i in range(5)]
color_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in color_array]
marker_colors=[]
for lat,lon,poi,cluster in zip(mum_merged['latitude'],mum_merged['longitude'],mum_merged['Neighborhood'],mum_merged['Cluster_Label']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=.7).add_to(map_cluster_mum)
map_cluster_mum



## Examining Cluster of Mumbai

#### Cluster 1

In [316]:
mum_merged.loc[mum_merged['Cluster_Label']==0,mum_merged.columns[[1]+list(range(5,mum_merged.shape[-1]))]]

,regionname,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
2,Mumbai,0,Indian Restaurant,Hotel,Coffee Shop,Roof Deck,Lounge,Chinese Restaurant,Restaurant,Gym / Fitness Center,Eastern European Restaurant,Electronics Store
3,Mumbai,0,Harbor / Marina,Indian Restaurant,Furniture / Home Store,Café,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store
4,Mumbai,0,Pharmacy,Pool Hall,Record Shop,Auto Garage,Zoo,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store
6,Mumbai,0,Historic Site,Soccer Field,Bus Station,Restaurant,Zoo,Doner Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,Mumbai,0,Gym,Rest Area,Convention Center,Creperie,Cretan Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
8,Mumbai,0,Café,Bar,Pub,Coffee Shop,Restaurant,Bistro,Italian Restaurant,Bookstore,Hotel,Pizza Place
9,Mumbai,0,Cocktail Bar,Bar,Café,Bakery,Coffee Shop,Park,Plaza,Pub,Bistro,Dance Studio
10,Mumbai,0,Restaurant,Gym,Hotel,Coffee Shop,Pub,Recreation Center,Café,Salon / Barbershop,Lounge,Beer Garden
11,Mumbai,0,Café,Coffee Shop,Pub,Bar,Bistro,Bakery,Plaza,Bookstore,Restaurant,Cocktail Bar
12,Mumbai,0,Camera Store,Gym,Tea Room,Bike Rental / Bike Share,Bus Station,Bakery,Coffee Shop,Athletics & Sports,Electronics Store,Drugstore


### Cluster 2

In [317]:
mum_merged.loc[mum_merged['Cluster_Label']==1,mum_merged.columns[[1]+list(range(5,mum_merged.shape[-1]))]]

,regionname,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
95,Mumbai,1,ATM,Women's Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
129,Mumbai,1,ATM,Fast Food Restaurant,Women's Store,Fish Market,Field,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
160,Mumbai,1,ATM,Women's Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
174,Mumbai,1,ATM,Women's Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 3 

In [319]:
mum_merged.loc[mum_merged['Cluster_Label']==2,mum_merged.columns[[1]+list(range(5,mum_merged.shape[-1]))]]

,regionname,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Mumbai,2,Indian Restaurant,Smoke Shop,Zoo,Doner Restaurant,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,Mumbai,2,Indian Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Chinese Restaurant,Café,Restaurant,BBQ Joint,Indian Sweet Shop,Dessert Shop,Electronics Store
5,Mumbai,2,Indian Restaurant,Movie Theater,Hockey Arena,Optical Shop,Zoo,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
31,Mumbai,2,Indian Restaurant,Mattress Store,Bus Station,Ice Cream Shop,Café,Zoo,Donut Shop,Field,Fast Food Restaurant,Farmers Market
35,Mumbai,2,Indian Restaurant,Snack Place,Fast Food Restaurant,Golf Course,Food Truck,Ice Cream Shop,Dog Run,Farmers Market,Farm,Falafel Restaurant
38,Mumbai,2,Vegetarian / Vegan Restaurant,Electronics Store,Dance Studio,Indian Restaurant,Zoo,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm
40,Mumbai,2,Indian Restaurant,Juice Bar,Gym / Fitness Center,Coffee Shop,Donut Shop,Restaurant,Electronics Store,Bed & Breakfast,Bakery,Men's Store
43,Mumbai,2,Indian Restaurant,Department Store,Multiplex,Chinese Restaurant,Lake,Hotel,Restaurant,Mughlai Restaurant,Eastern European Restaurant,Donut Shop
52,Mumbai,2,Indian Restaurant,Motorcycle Shop,Fast Food Restaurant,Hookah Bar,Hotel,Lounge,Doner Restaurant,Field,Farmers Market,Farm
73,Mumbai,2,Indian Restaurant,Café,Snack Place,Fast Food Restaurant,Train Station,Zoo,Farmers Market,Farm,Falafel Restaurant,Electronics Store


#### CLuster 4

In [321]:
mum_merged.loc[mum_merged['Cluster_Label']==3,mum_merged.columns[[1]+list(range(5,mum_merged.shape[-1]))]]

,regionname,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
145,Mumbai,3,Mobile Phone Shop,Fishing Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
175,Mumbai,3,Mobile Phone Shop,Fishing Store,Fish Market,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
199,Mumbai,3,Shopping Mall,Train Station,Mobile Phone Shop,Zoo,Doner Restaurant,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
204,Mumbai,3,Train Station,Zoo,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 5

In [322]:
mum_merged.loc[mum_merged['Cluster_Label']==4,mum_merged.columns[[1]+list(range(5,mum_merged.shape[-1]))]]

,regionname,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
165,Mumbai,4,Gym Pool,Zoo,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant


In [323]:
del_grouped_cluster=del_grouped.drop('Neighborhood',1)
model2=KMeans(n_clusters=5,random_state=5).fit(del_grouped_cluster)
model2.labels_

array([0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 3, 4, 2, 3, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 2, 0, 0])

In [324]:
neighborhood_venue_sorted_del.insert(0,'Cluster_label',model2.labels_)
del_merge=df_del
del_merge=del_merge.join(neighborhood_venue_sorted_del.set_index('Neighborhood'),on='Neighborhood')
del_merge.head()

,pincode,regionname,Neighborhood,latitude,longitude,Cluster_label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,110092,Delhi,"Anand Vihar S.O,IP Extension S.O,Laxmi Nagar S...",28.551696,77.417594,4.0,Clothing Store,Wings Joint,Farmers Market,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant
1,110051,Delhi,"Azad Nagar S.O (East Delhi),Govind Pura S.O,Kr...",28.653165,77.286960,0.0,Pizza Place,Accessories Store,Gym,Bakery,Fast Food Restaurant,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop
2,110032,Delhi,"Babarpur S.O (North East Delhi),Balbir Nagar S...",28.676479,77.285673,0.0,Light Rail Station,Park,Vegetarian / Vegan Restaurant,Wings Joint,Fast Food Restaurant,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop
3,110090,Delhi,"Badarpur Khadar B.O,Karawal Nagar S.O,Shaheed ...",-8.016630,-34.926281,0.0,Burger Joint,Bakery,Market,Snack Place,Gym / Fitness Center,Brazilian Restaurant,Supermarket,Pizza Place,Plaza,BBQ Joint
4,110053,Delhi,"Bhajan Pura S.O,Brahampuri S.O,Garhi Mandu B.O...",28.691913,77.279820,0.0,ATM,Clothing Store,IT Services,Mobile Phone Shop,Electronics Store,Park,Ice Cream Shop,American Restaurant,Flea Market,Frozen Yogurt Shop


In [325]:
del_merge=del_merge.dropna()

In [329]:
del_merge['Cluster_label']=del_merge.Cluster_label.astype(int)

In [331]:
map_cluster_del=folium.Map(location=[latitude_del,longitude_del],zoom_start=10)
x=np.arange(5)
ys=[i+x+i*x**2 for i in range(5)]
color_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in color_array]

marker_colors=[]
for lat,lon,poi,cluster in zip(del_merge['latitude'],del_merge['longitude'],del_merge['Neighborhood'],del_merge['Cluster_label']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster))
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=.7).add_to(map_cluster_del)
map_cluster_del

## Examining Clusters of Delhi 

#### Cluster 1

In [336]:
del_merge.loc[del_merge['Cluster_label']==0,del_merge.columns[[1]+list(range(5,del_merge.shape[-1]))]]

,regionname,Cluster_label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue,Cluster_Label
1,Delhi,0,Pizza Place,Accessories Store,Gym,Bakery,Fast Food Restaurant,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,0
2,Delhi,0,Light Rail Station,Park,Vegetarian / Vegan Restaurant,Wings Joint,Fast Food Restaurant,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,0
3,Delhi,0,Burger Joint,Bakery,Market,Snack Place,Gym / Fitness Center,Brazilian Restaurant,Supermarket,Pizza Place,Plaza,BBQ Joint,0
4,Delhi,0,ATM,Clothing Store,IT Services,Mobile Phone Shop,Electronics Store,Park,Ice Cream Shop,American Restaurant,Flea Market,Frozen Yogurt Shop,0
5,Delhi,0,Indian Restaurant,Snack Place,Gym / Fitness Center,Market,Bakery,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,0
7,Delhi,0,Indian Restaurant,Track,BBQ Joint,Athletics & Sports,Asian Restaurant,Juice Bar,Metro Station,Pool,Hotel,Shop & Service,0
8,Delhi,0,Pizza Place,Burger Joint,Park,Ice Cream Shop,Advertising Agency,Frozen Yogurt Shop,French Restaurant,Food Truck,Food Stand,Food Court,0
9,Delhi,0,Pizza Place,Convenience Store,Hotel,Stadium,Market,Fast Food Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,0
10,Delhi,0,ATM,Snack Place,Smoke Shop,Indian Restaurant,Hotel Pool,Hotel Bar,Food Court,Food & Drink Shop,Hotpot Restaurant,IT Services,0
11,Delhi,0,Indian Restaurant,Snack Place,Fast Food Restaurant,Dessert Shop,Market,Restaurant,Museum,Flea Market,Hardware Store,Food & Drink Shop,0


#### Cluster 2

In [337]:
del_merge.loc[del_merge['Cluster_label']==1,del_merge.columns[[1]+list(range(5,del_merge.shape[-1]))]]

,regionname,Cluster_label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue,Cluster_Label
20,Delhi,1,Electronics Store,Bank,Wings Joint,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,1
31,Delhi,1,Electronics Store,Pizza Place,Bank,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,1


#### Cluster 3

In [338]:
del_merge.loc[del_merge['Cluster_label']==2,del_merge.columns[[1]+list(range(5,del_merge.shape[-1]))]]

,regionname,Cluster_label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue,Cluster_Label
6,Delhi,2,ATM,Motorcycle Shop,Cultural Center,Fish & Chips Shop,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,2
12,Delhi,2,ATM,Light Rail Station,Electronics Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,2
21,Delhi,2,ATM,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,2
24,Delhi,2,ATM,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,2
28,Delhi,2,ATM,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,2
56,Delhi,2,ATM,Trail,Athletics & Sports,Indian Restaurant,Advertising Agency,Fruit & Vegetable Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,2
58,Delhi,2,ATM,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,2
78,Delhi,2,ATM,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,2
82,Delhi,2,ATM,Flower Shop,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,2


#### Cluster 4

In [339]:
del_merge.loc[del_merge['Cluster_label']==3,del_merge.columns[[1]+list(range(5,del_merge.shape[-1]))]]

,regionname,Cluster_label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue,Cluster_Label
49,Delhi,3,Fast Food Restaurant,Hotel,Chinese Restaurant,Seafood Restaurant,Wings Joint,Farmers Market,Food Truck,Food Stand,Food Court,Food & Drink Shop,3
75,Delhi,3,Fast Food Restaurant,Indian Restaurant,Coffee Shop,Café,Dessert Shop,Fish & Chips Shop,French Restaurant,Food Truck,Food Stand,Food Court,3
79,Delhi,3,Fast Food Restaurant,Train Station,Café,Electronics Store,Flea Market,Food,Candy Store,Vegetarian / Vegan Restaurant,Coffee Shop,Food & Drink Shop,3
86,Delhi,3,Fast Food Restaurant,Chinese Restaurant,Nature Preserve,Market,Fish & Chips Shop,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,3


#### Cluster 5

In [340]:
del_merge.loc[del_merge['Cluster_label']==4,del_merge.columns[[1]+list(range(5,del_merge.shape[-1]))]]

,regionname,Cluster_label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue,Cluster_Label
0,Delhi,4,Clothing Store,Wings Joint,Farmers Market,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,4
19,Delhi,4,Clothing Store,Asian Restaurant,Wings Joint,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,4
